In [1]:
import carla
import random
import Controller.PIDController as PIDController
import Controller.MPCController as MPCController
import math
import numpy as np
from synch_mode import CarlaSyncMode
import time
from utils import *

pygame 2.5.1 (SDL 2.28.2, Python 3.8.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import pygame

In [3]:
def wrap_angle(angle_in_degree):
    angle_in_rad = angle_in_degree / 180.0 * np.pi
    while (angle_in_rad > np.pi):
        angle_in_rad -= 2 * np.pi
    while (angle_in_rad <= -np.pi):
        angle_in_rad += 2 * np.pi
    return angle_in_rad

def get_vehicle_wheelbases(wheels, center_of_mass):
    front_left_wheel = wheels[0]
    front_right_wheel = wheels[1]
    back_left_wheel = wheels[2]
    back_right_wheel = wheels[3]
    front_x = (front_left_wheel.position.x + front_right_wheel.position.x) / 2.0
    front_y = (front_left_wheel.position.y + front_right_wheel.position.y) / 2.0
    front_z = (front_left_wheel.position.z + front_right_wheel.position.z) / 2.0
    back_x = (back_left_wheel.position.x + back_right_wheel.position.x) / 2.0
    back_y = (back_left_wheel.position.y + back_right_wheel.position.y) / 2.0
    back_z = (back_left_wheel.position.z + back_right_wheel.position.z) / 2.0
    l = np.sqrt( (front_x - back_x)**2 + (front_y - back_y)**2 + (front_z - back_z)**2  ) / 100.0
    # print(f"center of mass : {center_of_mass.x}, {center_of_mass.y}, {center_of_mass.z} wheelbase {l}")
    # return center_of_mass.x , l - center_of_mass.x, l
    return l - center_of_mass.x, center_of_mass.x, l

In [4]:
class HUD(object):
    def __init__(self):
        self.frame = 0
        self.simulation_time = 0


    def on_world_tick(self, timestamp):
        # self._server_clock.tick()
        # self.server_fps = self._server_clock.get_fps()
        self.frame = timestamp.frame
        self.simulation_time = timestamp.elapsed_seconds
    
    def get_simulation_information(self):
        return self.frame, self.simulation_time

    # def tick(self, world, clock):
    #     self._notifications.tick(world, clock)
    #     if not self._show_info:
    #         return
    #     t = world.player.get_transform()
    #     v = world.player.get_velocity()
    #     c = world.player.get_control()
    #     heading = 'N' if abs(t.rotation.yaw) < 89.5 else ''
    #     heading += 'S' if abs(t.rotation.yaw) > 90.5 else ''
    #     heading += 'E' if 179.5 > t.rotation.yaw > 0.5 else ''
    #     heading += 'W' if -0.5 > t.rotation.yaw > -179.5 else ''
    #     colhist = world.collision_sensor.get_collision_history()
    #     collision = [colhist[x + self.frame - 200] for x in range(0, 200)]
    #     max_col = max(1.0, max(collision))
    #     collision = [x / max_col for x in collision]
    #     vehicles = world.world.get_actors().filter('vehicle.*')
    #     self.record["speed"].append(math.sqrt(v.x**2 + v.y**2 + v.z**2))
    #     self.record["yaw"].append(t.rotation.yaw)
    #     self.record["time"].append(self.simulation_time)
    #     # print(self.simulation_time)
    #     pickle.dump(self.record, open( "MPC_record.pkl", "wb" ) )


    #     self._info_text = [
    #         'Server:  % 16.0f FPS' % self.server_fps,
    #         'Client:  % 16.0f FPS' % clock.get_fps(),
    #         '',
    #         'Vehicle: % 20s' % get_actor_display_name(world.player, truncate=20),
    #         'Map:     % 20s' % world.map.name,
    #         'Simulation time: % 12s' % datetime.timedelta(seconds=int(self.simulation_time)),
    #         '',
    #         'Speed:   % 15.0f km/h' % (3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2)),
    #         u'Heading:% 16.0f\N{DEGREE SIGN} % 2s' % (t.rotation.yaw, heading),
    #         'Location:% 30s' % ('(% 5.1f, % 5.1f, % 5.1f)' % (t.location.x, t.location.y, t.location.z)),
    #         'GNSS:% 24s' % ('(% 2.6f, % 3.6f)' % (world.gnss_sensor.lat, world.gnss_sensor.lon)),
    #         'Height:  % 18.0f m' % t.location.z,
    #         '']
    #     if isinstance(c, carla.VehicleControl):
    #         self._info_text += [
    #             ('Throttle:', c.throttle, 0.0, 1.0),
    #             ('Steer:', c.steer, -1.0, 1.0),
    #             ('Brake:', c.brake, 0.0, 1.0),
    #             ('Reverse:', c.reverse),
    #             ('Hand brake:', c.hand_brake),
    #             ('Manual:', c.manual_gear_shift),
    #             'Gear:        %s' % {-1: 'R', 0: 'N'}.get(c.gear, c.gear)]
    #     elif isinstance(c, carla.WalkerControl):
    #         self._info_text += [
    #             ('Speed:', c.speed, 0.0, 5.556),
    #             ('Jump:', c.jump)]
    #     self._info_text += [
    #         '',
    #         'Collision:',
    #         collision,
    #         '',
    #         'Number of vehicles: % 8d' % len(vehicles)]
    #     if len(vehicles) > 1:
    #         self._info_text += ['Nearby vehicles:']
    #         distance = lambda l: math.sqrt((l.x - t.location.x)**2 + (l.y - t.location.y)**2 + (l.z - t.location.z)**2)
    #         vehicles = [(distance(x.get_location()), x) for x in vehicles if x.id != world.player.id]
    #         for d, vehicle in sorted(vehicles):
    #             if d > 200.0:
    #                 break
    #             vehicle_type = get_actor_display_name(vehicle, truncate=22)
    #             self._info_text.append('% 4dm %s' % (d, vehicle_type))



In [5]:
import argparse

argparser = argparse.ArgumentParser(
    description='CARLA Manual Control Client')
argparser.add_argument(
    '-v', '--verbose',
    action='store_true',
    dest='debug',
    help='print debug information')
argparser.add_argument(
    '--host',
    metavar='H',
    default='127.0.0.1',
    help='IP of the host server (default: 127.0.0.1)')
argparser.add_argument(
    '-p', '--port',
    metavar='P',
    default=2000,
    type=int,
    help='TCP port to listen to (default: 2000)')
argparser.add_argument(
    '-a', '--autopilot',
    action='store_true',
    help='enable autopilot')
argparser.add_argument(
    '--res',
    metavar='WIDTHxHEIGHT',
    default='1280x720',
    help='window resolution (default: 1280x720)')
argparser.add_argument(
    '--filter',
    metavar='PATTERN',
    default='vehicle.*',
    help='actor filter (default: "vehicle.*")')
argparser.add_argument(
    '--rolename',
    metavar='NAME',
    default='hero',
    help='actor role name (default: "hero")')
argparser.add_argument(
    '--gamma',
    default=2.2,
    type=float,
    help='Gamma correction of the camera (default: 2.2)')
argparser.add_argument(
    '--map',
    metavar='NAME',
    default='Town04',
    help='simulation map (default: "Town04")')
argparser.add_argument(
    '--spawn_x',
    metavar='x',
    default='-16.75', #town04 = -16.75
    help='x position to spawn the agent')
argparser.add_argument(
    '--spawn_y',
    metavar='y',
    default='-223.55', #town04 = -223.55
    help='y position to spawn the agent')
argparser.add_argument(
    '--random_spawn',        
    metavar='RS',
    default='0',
    type=int,
    help='Random spawn agent')
argparser.add_argument(
    '--vehicle_id',
    metavar='NAME',
    # default='vehicle.jeep.wrangler_rubicon',
    default='vehicle.tesla.model3',
    help='vehicle to spawn, available options : vehicle.audi.a2 vehicle.audi.tt vehicle.carlamotors.carlacola vehicle.citroen.c3 vehicle.dodge_charger.police vehicle.jeep.wrangler_rubicon vehicle.yamaha.yzf vehicle.nissan.patrol vehicle.gazelle.omafiets vehicle.bh.crossbike vehicle.ford.mustang vehicle.bmw.isetta vehicle.audi.etron vehicle.harley-davidson.low rider vehicle.mercedes-benz.coupe vehicle.bmw.grandtourer vehicle.toyota.prius vehicle.diamondback.century vehicle.tesla.model3 vehicle.seat.leon vehicle.lincoln.mkz2017 vehicle.kawasaki.ninja vehicle.volkswagen.t2 vehicle.nissan.micra vehicle.chevrolet.impala vehicle.mini.cooperst')
argparser.add_argument(
    '--vehicle_wheelbase',
    metavar='NAME',
    type=float,
    default='2.89',
    help='vehicle wheelbase used for model predict control')
argparser.add_argument(
    '--waypoint_resolution',
    metavar='WR',
    default='0.5',
    type=float,
    help='waypoint resulution for control')
argparser.add_argument(
    '--waypoint_lookahead_distance',
    metavar='WLD',
    default='5.0',
    type=float,
    help='waypoint look ahead distance for control')
argparser.add_argument(
    '--desired_speed',
    metavar='SPEED',
    default='13.89',
    type=float,
    help='desired speed for highway driving')
argparser.add_argument(
    '--control_mode',
    metavar='CONT',
    default='MPC',
    help='Controller')
argparser.add_argument(
    '--planning_horizon',
    metavar='HORIZON',
    type=int,
    default='3',
    help='Planning horizon for MPC')
argparser.add_argument(
    '--time_step',
    metavar='DT',
    default='0.4',
    type=float,
    help='Planning time step for MPC')
argparser.add_argument(
    '--FPS',
    metavar='FPS',
    default='10',
    type=int,
    help='Frame per second for simulation')

args = argparser.parse_args()

args.width, args.height = [int(x) for x in args.res.split('x')]

In [6]:
class World(object):
    def __init__(self, client, carla_world, hud, args):
        self.world = carla_world
        self.client = client
        self.actor_role_name = args.rolename
        self.map = self.world.get_map()
        self.hud = hud
        self.player = None
        self.collision_sensor = None
        self.lane_invasion_sensor = None
        self.gnss_sensor = None
        self.camera_manager = None
        # self._weather_presets = find_weather_presets()
        self._weather_index = 0
        self._actor_filter = "vehicle.*"
        self._gamma = 2.2
        self.args = args
        self.recording_start = 0
        self.waypoint_resolution = 0.5
        self.waypoint_lookahead_distance = 5
        self.desired_speed = 14
        self.planning_horizon = 3
        self.time_step = 0.4
        self.control_mode = 'MPC'
        self.controller = None
        self.control_count = 0.0
        self.random_spawn = 0
        self.restart()
        self.world.on_tick(hud.on_world_tick)
        self.recording_enabled = False
        self.im_width = 640
        self.im_height = 480
        self.episode_start = 0
        self.visuals = False

        if self.visuals:
            self._initiate_visuals()
        
    def restart(self):
        # Keep same camera config if the camera manager exists.
        cam_index = self.camera_manager.index if self.camera_manager is not None else 0
        cam_pos_index = self.camera_manager.transform_index if self.camera_manager is not None else 0
        # Get a random blueprint.
        blueprints = self.world.get_blueprint_library().filter(self._actor_filter)
        blueprint = None
        for blueprint_candidates in blueprints:
            # print(blueprint_candidates.id)
            if blueprint_candidates.id == self.args.vehicle_id:
                blueprint = blueprint_candidates
                break
        if blueprint is None:
            blueprint = random.choice(self.world.get_blueprint_library().filter(self._actor_filter))

        blueprint.set_attribute('role_name', self.actor_role_name)
        if blueprint.has_attribute('color'):
            color = random.choice(blueprint.get_attribute('color').recommended_values)
            blueprint.set_attribute('color', color)
        if blueprint.has_attribute('driver_id'):
            driver_id = random.choice(blueprint.get_attribute('driver_id').recommended_values)
            blueprint.set_attribute('driver_id', driver_id)
        if blueprint.has_attribute('is_invincible'):
            blueprint.set_attribute('is_invincible', 'true')
        # Spawn the player.
        if self.player is not None:
            spawn_point = self.player.get_transform()
            spawn_point.location.z += 2.0
            spawn_point.rotation.roll = 0.0
            spawn_point.rotation.pitch = 0.0
            self.destroy()
            self.player = self.world.try_spawn_actor(blueprint, spawn_point)

        while self.player is None:
            if self.random_spawn == 1:
                print(f"Random spawn!")
                spawn_points = self.world.get_map().get_spawn_points()
                spawn_point = random.choice(spawn_points) if spawn_points else carla.Transform()
                self.player = self.world.try_spawn_actor(blueprint, spawn_point)
            else:
                spawn_location = carla.Location()
                spawn_location.x = float(self.args.spawn_x)
                spawn_location.y = float(self.args.spawn_y)
                spawn_waypoint = self.map.get_waypoint(spawn_location)
                spawn_transform = spawn_waypoint.transform
                spawn_transform.location.z = 1.0
                self.player = self.world.try_spawn_actor(blueprint, spawn_transform)

            spectator = self.world.get_spectator()
            transform = self.player.get_transform()
            spectator.set_transform(carla.Transform(transform.location + carla.Location(y=-10,z=28.5), carla.Rotation(pitch=-90)))

            # spawn_points = self.world.get_map().get_spawn_points()
            # spawn_point = random.choice(spawn_points) if spawn_points else carla.Transform()
            # self.player = self.world.try_spawn_actor(blueprint, spawn_point)

            self.blueprint_library = self.world.get_blueprint_library()
            self.vehicle_blueprint = self.blueprint_library.filter('*vehicle*')
            self.walker_blueprint = self.blueprint_library.filter('*walker.*')

            ## CAMERA RGB

            self.rgb_cam = self.blueprint_library.find('sensor.camera.rgb')
            self.rgb_cam.set_attribute("image_size_x", f"{640}")
            self.rgb_cam.set_attribute("image_size_y", f"{480}")
            self.rgb_cam.set_attribute("fov", f"110")
            self.camera_rgb = self.world.spawn_actor(
                self.rgb_cam,
                carla.Transform(carla.Location(x=2, z=1), carla.Rotation(0,0,0)),
                attach_to=self.player)

            ## CAMERA FOR VIZUALIZATION

            self.vis_cam = self.blueprint_library.find('sensor.camera.rgb')
            self.vis_cam.set_attribute("image_size_x", f"{640}")
            self.vis_cam.set_attribute("image_size_y", f"{480}")
            self.vis_cam.set_attribute("fov", f"110")
            self.camera_rgb_vis = self.world.spawn_actor(
                self.vis_cam,
                carla.Transform(carla.Location(x=-5.5, z=2.8), carla.Rotation(pitch=-15)),
                attach_to=self.player)

            ## LANE VIZUALIZATION

            self.lane_invasion = self.world.spawn_actor(
                self.blueprint_library.find('sensor.other.lane_invasion'), 
                carla.Transform(), 
                attach_to=self.player)

            ## COLLISION SENSOR

            self.collision_sensor = self.world.spawn_actor(
                self.blueprint_library.find('sensor.other.collision'),
                carla.Transform(),
                attach_to=self.player)
            
            ## CONTROLLER

            self.control_count = 0
            if self.control_mode == "PID":
                self.controller = PIDController.Controller()
            elif self.control_mode == "MPC":
                physic_control = self.player.get_physics_control()
                lf, lr, l = get_vehicle_wheelbases(physic_control.wheels, physic_control.center_of_mass )
                self.controller = MPCController.Controller(lf = lf, lr = lr, wheelbase=l, planning_horizon = self.planning_horizon, time_step = self.time_step)
            velocity_vec = self.player.get_velocity()
            current_transform = self.player.get_transform()
            current_location = current_transform.location
            current_roration = current_transform.rotation
            current_x = current_location.x
            current_y = current_location.y
            current_yaw = wrap_angle(current_roration.yaw)
            current_speed = math.sqrt(velocity_vec.x**2 + velocity_vec.y**2 + velocity_vec.z**2)
            frame, current_timestamp = self.hud.get_simulation_information()
            print(frame)
            print(current_timestamp)
            self.controller.update_values(current_x, current_y, current_yaw, current_speed, current_timestamp, frame)

    def tick(self, clock):
        self.hud.tick(self, clock)

    def destroy_sensors(self):
        self.camera_manager.sensor.destroy()
        self.camera_manager.sensor = None
        self.camera_manager.index = None

    def destroy(self):
        actors = [
            self.player,
            self.collision_sensor,
            self.camera_rgb,
            self.camera_rgb_vis,
            self.lane_invasion
                    ]
        for actor in actors:
            if actor is not None:
                actor.destroy()



    def _initiate_visuals(self):
        pygame.init()

        self.display = pygame.display.set_mode(
            (800, 600),
            pygame.HWSURFACE | pygame.DOUBLEBUF)
        self.font = get_font()
        self.clock = pygame.time.Clock()



    def generate_episode(self, controller):


        with CarlaSyncMode(self.world, self.camera_rgb, self.camera_rgb_vis, self.lane_invasion, self.collision_sensor, fps=30) as sync_mode:
            

            counter = 0
            snapshot, image_rgb, image_rgb_vis, lane, collision = sync_mode.tick(timeout=10.0)
            self.episode_start = time.time()
            # vehicle_location = self.player.get_location()
            # print(vehicle_location)


            # image = process_img2(self,image_rgb)
            # next_state = image 

            # controller = VehicleControl(self.world)
            clock = pygame.time.Clock()
            while True:
                
                # if self.visuals:
                #     if should_quit():
                #         return
                #     self.clock.tick_busy_loop(30)


                # clock.tick(5)
                clock.tick_busy_loop(args.FPS)
                # pygame.time.dalay(500)
                if controller.parse_events(self.client, self.world, clock):
                    return
                snapshot, image_rgb, image_rgb_vis, lane, collision = sync_mode.tick(timeout=10.0)
                # world.tick(clock)
                # time.sleep (30)

        
   

In [7]:
class VehicleControl(object):
    def __init__(self, world):
        self.world = world
        self._autopilot_enabled = False
        self._control = carla.VehicleControl()
        # if isinstance(self.world.player, carla.Vehicle):
        #     self._control = carla.VehicleControl()
        #     self.world.player.set_autopilot(self._autopilot_enabled)
        # elif isinstance(self.world.player, carla.Walker):
        #     self._control = carla.WalkerControl()
        #     self._autopilot_enabled = False
        #     self._rotation = self.world.player.get_transform().rotation
        # else:
            # raise NotImplementedError("Actor type not supported")
        self._steer_cache = 0.0
        # world.hud.notification("Press 'H' or '?' for help.", seconds=4.0)

    def parse_events(self, client, world, clock):
        if not self._autopilot_enabled:
            # Control loop
            # get waypoints
            current_location = self.world.player.get_location()
            velocity_vec = self.world.player.get_velocity()
            current_transform = self.world.player.get_transform()
            current_location = current_transform.location
            current_rotation = current_transform.rotation
            current_x = current_location.x
            current_y = current_location.y
            current_yaw = wrap_angle(current_rotation.yaw)
            current_speed = math.sqrt(velocity_vec.x**2 + velocity_vec.y**2 + velocity_vec.z**2)
            print(f"Control input : speed : {current_speed}, current position : {current_x}, {current_y}, yaw : {current_yaw}")
            frame, current_timestamp =self.world.hud.get_simulation_information()
            ready_to_go = self.world.controller.update_values(current_x, current_y, current_yaw, current_speed, current_timestamp, frame)
            if ready_to_go:
                if self.world.control_mode == "PID":
                    current_location = self.world.player.get_location()
                    current_waypoint = self.world.map.get_waypoint(current_location).next(self.world.waypoint_resolution)[0]
                    waypoints = []
                    for i in range(int(self.world.waypoint_lookahead_distance / self.world.waypoint_resolution)):
                        waypoints.append([current_waypoint.transform.location.x, current_waypoint.transform.location.y, self.world.desired_speed])
                        current_waypoint = current_waypoint.next(self.world.waypoint_resolution)[0]
                    # print(f"number of waypoints : {len(next_waypoint_list)}")
                    # current_location = world.player.get_location()
                elif self.world.control_mode == "MPC":
                    dist = self.world.time_step * current_speed + 0.1
                    prev_waypoint = self.world.map.get_waypoint(current_location)
                    current_waypoint = prev_waypoint.next(dist)[0]
                    print(current_waypoint)
                    waypoints = []
                    road_desired_speed = self.world.desired_speed
                    # road_desired_speed = world.player.get_speed_limit()/3.6*0.95
                    for i in range(self.world.planning_horizon):
                        if self.world.control_count + i <= 100:
                            desired_speed = (self.world.control_count + 1 + i)/100.0 * road_desired_speed
                        else:
                            desired_speed = road_desired_speed
                        dist = self.world.time_step * road_desired_speed
                        current_waypoint = prev_waypoint.next(dist)[0]
                        waypoints.append([current_waypoint.transform.location.x, current_waypoint.transform.location.y, road_desired_speed, wrap_angle(current_waypoint.transform.rotation.yaw)])
                        prev_waypoint = current_waypoint
                
                
                print(waypoints)
                print(current_x-waypoints[0][0])
                print(current_x-waypoints[1][0])
                print(current_x-waypoints[2][0])

                print(current_y-waypoints[0][1])
                print(current_y-waypoints[1][1])
                print(current_y-waypoints[2][1])

             
              
                self.world.controller.update_waypoints(waypoints)     
                self.world.controller.update_controls()
                self._control.throttle, self._control.steer, self._control.brake = self.world.controller.get_commands()
                self.world.player.apply_control(self._control)
                self.world.control_count += 1
            # world.player.set_transform(current_waypoint.transform)


In [8]:
def game_loop(args): 
    world=None   
    try: 
        client = carla.Client(args.host, args.port)
        client.set_timeout(100.0)
        hud = HUD()
        # carla_world = client.load_world(args.map)
        carla_world = client.get_world()
        world = World(carla_world, hud, args)
        controller = VehicleControl(world)

        clock = pygame.time.Clock()
        while True:
            # clock.tick(5)
            clock.tick_busy_loop(args.FPS)
            # pygame.time.dalay(500)
            if controller.parse_events(client, world, clock):
                return
            # world.tick(clock)

    finally:

            if world is not None:
                world.destroy()        

In [9]:
client = carla.Client(args.host, args.port)
client.set_timeout(100.0)

In [10]:
carla_world = client.load_world(args.map)

In [10]:
carla_world = client.get_world()

In [11]:
hud = HUD()

In [12]:
world = World(client, carla_world, hud, args)

0
0


In [13]:
controller = VehicleControl(world)

In [14]:
try:   
    world.generate_episode(controller)

finally:

    if world is not None:
        world.destroy()   

Control input : speed : 0.0, current position : -16.893173217773438, -223.48516845703125, yaw : 1.5896144893735056
Waypoint(Transform(Location(x=-16.896622, y=-223.301270, z=0.000000), Rotation(pitch=360.000000, yaw=91.018341, roll=0.000000)))
[[-16.91511344909668, -217.45973205566406, 14, 1.5668718926125558], [-16.89313507080078, -211.85977172851562, 14, 1.5668718926125558], [-16.871158599853516, -206.2598114013672, 14, 1.5668718926125558]]
0.021940231323242188
-3.814697265625e-05
-0.022014617919921875
-6.0254364013671875
-11.625396728515625
-17.225357055664062
Control input : speed : 0.0015572347848653194, current position : -16.893173217773438, -223.48516845703125, yaw : 1.5896144893735056
Waypoint(Transform(Location(x=-16.896637, y=-223.300583, z=0.000000), Rotation(pitch=360.000000, yaw=91.018120, roll=0.000000)))
[[-16.91511344909668, -217.45973205566406, 14, 1.5668718926125558], [-16.89313507080078, -211.85977172851562, 14, 1.5668718926125558], [-16.871158599853516, -206.2598114

KeyboardInterrupt: 

In [22]:
world.destroy() 

RuntimeError: trying to operate on a destroyed actor; an actor's function was called, but the actor is already destroyed.

In [41]:
try:

    game_loop(args)

except KeyboardInterrupt:
    print('\nCancelled by user. Bye!')

0
0
Control input : speed : 1.5275484323501587, current position : -16.893253326416016, -223.4851837158203, yaw : 1.5896147556896145
Waypoint(Transform(Location(x=-16.907263, y=-222.630295, z=0.000000), Rotation(pitch=360.000000, yaw=90.799843, roll=0.000000)))
[[-16.91511344909668, -217.45974731445312, 14, 1.5668718926125558], [-16.89313507080078, -211.8597869873047, 14, 1.5668718926125558], [-16.871158599853516, -206.25982666015625, 14, 1.5668718926125558]]
0.021860122680664062
-0.000118255615234375
-0.0220947265625
-6.0254364013671875
-11.625396728515625
-17.225357055664062
Control input : speed : 2.5621190071105957, current position : -16.893253326416016, -223.4851837158203, yaw : 1.5896147556896145
Waypoint(Transform(Location(x=-16.913029, y=-222.175827, z=0.000000), Rotation(pitch=360.000000, yaw=450.651825, roll=0.000000)))
[[-16.91511344909668, -217.45974731445312, 14, 1.5668718926125558], [-16.89313507080078, -211.8597869873047, 14, 1.5668718926125558], [-16.871158599853516, -

In [5]:
# Retrieve the spectator object
spectator = world.get_spectator()

# Get the location and rotation of the spectator through its transform
transform = spectator.get_transform()

location = transform.location
rotation = transform.rotation

In [8]:
## get spectator location.

t = world.get_spectator().get_transform()
# coordinate_str = "(x,y) = ({},{})".format(t.location.x, t.location.y)
coordinate_str = "(x,y,z) = ({},{},{})".format(t.location.x, t.location.y,t.location.z)
print (coordinate_str)

(x,y,z) = (-16.747251510620117,-223.55465698242188,1.4331684112548828)
